In [1]:
import cmath
from copy import copy, deepcopy
import itertools
import warnings

from FuncClasses import *
from utils import *

import numpy as np
import sympy as sym
import scipy as sp
from scipy.optimize import NonlinearConstraint, Bounds, minimize, linprog, OptimizeWarning

# Functions

## Scipy

In [2]:
def diff_fsum(fsum, i):
    new_fsum = []
    for fterm in fsum.fterms:
        t = deepcopy(fterm.var)
        if t[i] != 0:
            c = fterm.coeff * t[i]
            t[i] -= 1
            new_fsum.append(FuncTerm(c, t))
    return FuncSum(new_fsum)

def scipy_find_b(barrier, states, f_vec, term_powers, prec=2, linprog_obj=[]):
    # Make appropriate conditions using representation
    dbdz = FuncVec([diff_fsum(barrier, i) for i in range(states)])
    dbdzconj = FuncVec([diff(barrier, i) for i in range(states, 2*states)])
    dbdt = (dbdz * f_vec) + (dbdzconj * f_vec.conj())
    barr_minus_conj = barrier - barrier.conj()
    
    # Create linprog matrices and vectors
    A_dbdt = [list(term.coeff.imag) for term in dbdt.fterms]
    b_dbdt = [0]*len(A_dbdt)
    
    A_conj = [list(term.coeff) for term in barr_minus_conj.fterms]
    b_conj = [0]*len(A_conj)
    
    A = np.array(A_dbdt + A_conj)
    b = np.array(b_dbdt + b_conj)
    # Minimize vector (min c^T x subject to ...)
    c = np.array([0]*len(term_powers))
    for i in linprog_obj:
        c[i] = 1
    
    # Negate A doesn't matter unless used for A_ub
    res = linprog(c,
                  A_eq=A,
                  b_eq=b,
                  bounds=(-1,1),
                 )
    return barrier.get_sym_sum(res.x, prec)

In [3]:
def scipy_find_constant(barrier_sym, states, init=[], prec=2):
    # Create objective function from Sympy barrier
    def obj(x):
        z = [x[2*i] + 1j*x[2*i + 1] for i in range(states)]
        b = barrier_sym
        for var_sym in barrier_sym.free_symbols:
            sym_num = int(str(var_sym)[1:])
            b = b.subs({var_sym: z[sym_num]})
        return -float(sym.re(b))

    # Bounds and guesses
    bounds = Bounds([-1]*2*states, [1]*2*states)
    guess = [0]*2*states
    
    res = minimize(obj,
                   guess,
                   method='trust-constr',
                   constraints=init,
                   options={'verbose': 0},
                   bounds=bounds,
                   hess=lambda x: np.zeros((2*states,))
                  )
    minimum = round(res.fun, prec)
    return minimum

In [4]:
def scipy_check_constant(c, barrier_sym, states, unsafe=[], prec=2):
    def obj(x):
        z = [x[2*i] + 1j*x[2*i + 1] for i in range(states)]
        b = barrier_sym
        for var_sym in barrier_sym.free_symbols:
            sym_num = int(str(var_sym)[1:])
            b = b.subs({var_sym: z[sym_num]})
        return float(sym.re(b))

    # Bounds and guesses
    bounds = Bounds([-1]*2*states, [1]*2*states)
    guess = [0]*2*states
    
    res = minimize(obj,
                   guess,
                   method='trust-constr',
                   constraints=unsafe,
                   options={'verbose': 0},
                   bounds=bounds,
                   hess=lambda x: np.zeros((2*states,))
                  )
    minimum = round(res.fun, prec)
    if -minimum >= c :
        raise Exception(str(barrier_sym) + "\nError: proposed barrier has part of unsafe in same contour as initial region")

In [5]:
# Setup coefficients as ndarray
def scipy_find_k_barrier(k, H, init=[], unsafe=[], linprog_obj = [], prec=2, verbose=False):
    z = -1j
    n = round(len(H))
    term_powers = generate_term_powers(k, n)
    coeff_num = len(term_powers)
    

    if verbose: print("Converting dynamical system...")
    sums = []
    for i in range(n):
        terms = []
        for j in range(n):
            t = [0]*(2*n)
            t[j] = 1
            t = tuple(t)
            terms.append(FuncTerm(z * H[i][j], t))
        sums.append(FuncSum(terms))
    f_vec = FuncVec(sums)
    if verbose: print("Dynamical system converted.")
    
    id_coeff = np.identity(coeff_num)
    barrier = FuncSum(list([FuncTerm(i, t) for i, t in zip(id_coeff, term_powers)]))
    if verbose: print("Finding polynomial...")
    warnings.simplefilter("ignore", OptimizeWarning)
    b = scipy_find_b(barrier, n, f_vec, term_powers, prec, linprog_obj)
    if verbose: print("Polynomial found: ", b)
    
    if verbose: print("Finding constant...")
    c = scipy_find_constant(b, n, init=init)
    if verbose: print("Checking...")
    scipy_check_constant(c, b, n, unsafe=unsafe)
    if verbose: print("Constant found: ", c)
    
    return round_sympy_expr(c + b)

# Using Scipy

In [6]:
def qubit_simple_constraints(zero_bounds, one_bounds):
    def f(x): return [x[0]**2 + x[1]**2,
                      x[2]**2 + x[3]**2,
                      x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2]
    return [NonlinearConstraint(f, [zero_bounds[0], one_bounds[0], 1], [zero_bounds[1], one_bounds[1], 1])]

close_to_zero = qubit_simple_constraints((0.9,1),(0,0.1))
close_to_one = qubit_simple_constraints((0,0.1),(0.9,1))

## $e^{-i\hat{H}t} = H$

See https://doi.org/10.1088/1361-6455/aa987c

In [7]:
# Hadamard as a Hamiltonian
# H = [[1/2*np.sqrt(2), 1/2*np.sqrt(2)],[1/2*np.sqrt(2), -1/2*np.sqrt(2)]]
H = [[1,1],[1,-1]]
# For Had-like Hadamard
# c[12] = 1

init_constraints = close_to_zero
unsafe_constraints = qubit_simple_constraints((0,0.1),(0.9,1))


barrier = scipy_find_k_barrier(2, H,
                               init=init_constraints,
                               unsafe=unsafe_constraints,
                               linprog_obj=[12],
                               verbose=1)
barrier

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  -1.0*z0*conjugate(z0) - 0.33*z0*conjugate(z1) - 0.33*z1*conjugate(z0) - 0.33*z1*conjugate(z1)
Finding constant...
Checking...
Constant found:  0.74


-1.0*z0*conjugate(z0) - 0.33*z0*conjugate(z1) - 0.33*z1*conjugate(z0) - 0.33*z1*conjugate(z1) + 0.74

3 * Barrier $\approx \displaystyle - 3.0 z_{0} \overline{z_{0}} - 1.0 z_{0} \overline{z_{1}} - 1.0 z_{1} \overline{z_{0}} - 1.0 z_{1} \overline{z_{1}} + 2.2$


In Bra-ket notation: $ \displaystyle 2.2 -3\langle 0| \phi \rangle \langle \phi| 0\rangle - \langle 0| \phi\rangle \langle \phi|1 \rangle - \langle 1|\phi \rangle \langle \phi|0 \rangle - \langle 1|\phi \rangle \langle \phi| 1\rangle$

## $e^{-iHt} = S$

$S = \begin{bmatrix} 1 & 0 \\ 0 & i \end{bmatrix} $

In [8]:
z = -1j
n = 2
k = 2
term_powers = generate_term_powers(k, n)
for i in range(len(term_powers)):
    print(i, term_powers[i])

0 [0, 0, 0, 0]
1 [0, 0, 0, 1]
2 [0, 0, 0, 2]
3 [0, 0, 1, 0]
4 [0, 0, 1, 1]
5 [0, 0, 2, 0]
6 [0, 1, 0, 0]
7 [0, 1, 0, 1]
8 [0, 1, 1, 0]
9 [0, 2, 0, 0]
10 [1, 0, 0, 0]
11 [1, 0, 0, 1]
12 [1, 0, 1, 0]
13 [1, 1, 0, 0]
14 [2, 0, 0, 0]


In [9]:
d = np.pi/4
d=1
H = [[d,0], [0,-d]]
init_constraints = close_to_zero
unsafe_constraints = qubit_simple_constraints((0,np.sqrt(0.89)),(np.sqrt(0.11),1))

barrier = scipy_find_k_barrier(2, H,
                               init=init_constraints,
                               unsafe=unsafe_constraints,
                               linprog_obj=[12],
                               verbose=1)
barrier

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  -1.0*z0*conjugate(z0)
Finding constant...
Checking...
Constant found:  0.9


-1.0*z0*conjugate(z0) + 0.9

$0.9 - 1.0 z_0 \overline{z_0}$

In [10]:
d = np.pi/4
d=1
H = [[d,0], [0,-d]]
init_constraints = close_to_one
unsafe_constraints = qubit_simple_constraints((np.sqrt(0.11),1),(0,np.sqrt(0.89)))

barrier = scipy_find_k_barrier(2, H,
                               init=init_constraints,
                               unsafe=unsafe_constraints,
                               linprog_obj=[7],
                               verbose=1)
barrier

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  -1.0*z1*conjugate(z1)
Finding constant...
Checking...
Constant found:  0.9


-1.0*z1*conjugate(z1) + 0.9

$0.9 - 1.0 z_1 \overline{z_1}$

### Phase with slight disturbance

In [11]:
d = np.pi/4
d=1
H = [[d,0.1], [0.1,-d]]
init_constraints = close_to_one
unsafe_constraints = qubit_simple_constraints((np.sqrt(0.11),1),(0,np.sqrt(0.89)))

barrier = scipy_find_k_barrier(2, H,
                               init=init_constraints,
                               unsafe=unsafe_constraints,
                               linprog_obj=[7],
                               verbose=1)
barrier

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  0.05*z0*conjugate(z1) + 0.05*z1*conjugate(z0) - 1.0*z1*conjugate(z1)
Finding constant...
Checking...
Constant found:  0.87


0.05*z0*conjugate(z1) + 0.05*z1*conjugate(z0) - 1.0*z1*conjugate(z1) + 0.87

### Control small region

Control above

In [12]:
d = np.pi/4
d=1
H = [[d,0], [0,-d]]
init_constraints = qubit_simple_constraints((np.sqrt(0.4),np.sqrt(0.6)),(np.sqrt(0.4),np.sqrt(0.6)))
unsafe_constraints = qubit_simple_constraints((np.sqrt(0.7),1),(0,np.sqrt(0.3)))

barrier = scipy_find_k_barrier(2, H,
                               init=init_constraints,
                               unsafe=unsafe_constraints,
                               linprog_obj=[7],
                               verbose=1)
barrier

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  -1.0*z1*conjugate(z1)
Finding constant...
Checking...
Constant found:  0.54


-1.0*z1*conjugate(z1) + 0.54

$0.54 -1.0 z_1 \overline{z_1}$

Control below

In [13]:
d = np.pi/4
d=1
H = [[d,0], [0,-d]]
init_constraints = qubit_simple_constraints((np.sqrt(0.4),np.sqrt(0.6)),(np.sqrt(0.4),np.sqrt(0.6)))
unsafe_constraints = qubit_simple_constraints((0,np.sqrt(0.3)),(np.sqrt(0.7),1))

barrier = scipy_find_k_barrier(2, H,
                               init=init_constraints,
                               unsafe=unsafe_constraints,
                               linprog_obj=[12],
                               verbose=1)
barrier

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  -1.0*z0*conjugate(z0)
Finding constant...
Checking...
Constant found:  0.54


-1.0*z0*conjugate(z0) + 0.54

$0.54 -1.0 z_0 \overline{z_0}$

## $e^{-iHt} = CNOT$

In [14]:
z = -1j
n = 4
k = 2
term_powers = generate_term_powers(k, n)
for i in range(len(term_powers)):
    print(i, term_powers[i])

0 [0, 0, 0, 0, 0, 0, 0, 0]
1 [0, 0, 0, 0, 0, 0, 0, 1]
2 [0, 0, 0, 0, 0, 0, 0, 2]
3 [0, 0, 0, 0, 0, 0, 1, 0]
4 [0, 0, 0, 0, 0, 0, 1, 1]
5 [0, 0, 0, 0, 0, 0, 2, 0]
6 [0, 0, 0, 0, 0, 1, 0, 0]
7 [0, 0, 0, 0, 0, 1, 0, 1]
8 [0, 0, 0, 0, 0, 1, 1, 0]
9 [0, 0, 0, 0, 0, 2, 0, 0]
10 [0, 0, 0, 0, 1, 0, 0, 0]
11 [0, 0, 0, 0, 1, 0, 0, 1]
12 [0, 0, 0, 0, 1, 0, 1, 0]
13 [0, 0, 0, 0, 1, 1, 0, 0]
14 [0, 0, 0, 0, 2, 0, 0, 0]
15 [0, 0, 0, 1, 0, 0, 0, 0]
16 [0, 0, 0, 1, 0, 0, 0, 1]
17 [0, 0, 0, 1, 0, 0, 1, 0]
18 [0, 0, 0, 1, 0, 1, 0, 0]
19 [0, 0, 0, 1, 1, 0, 0, 0]
20 [0, 0, 0, 2, 0, 0, 0, 0]
21 [0, 0, 1, 0, 0, 0, 0, 0]
22 [0, 0, 1, 0, 0, 0, 0, 1]
23 [0, 0, 1, 0, 0, 0, 1, 0]
24 [0, 0, 1, 0, 0, 1, 0, 0]
25 [0, 0, 1, 0, 1, 0, 0, 0]
26 [0, 0, 1, 1, 0, 0, 0, 0]
27 [0, 0, 2, 0, 0, 0, 0, 0]
28 [0, 1, 0, 0, 0, 0, 0, 0]
29 [0, 1, 0, 0, 0, 0, 0, 1]
30 [0, 1, 0, 0, 0, 0, 1, 0]
31 [0, 1, 0, 0, 0, 1, 0, 0]
32 [0, 1, 0, 0, 1, 0, 0, 0]
33 [0, 1, 0, 1, 0, 0, 0, 0]
34 [0, 1, 1, 0, 0, 0, 0, 0]
35 [0, 2, 0, 0, 0, 0, 0, 0]
36

In [28]:
# H = [[0,0,0,0],[0,0,0,0],[0,0,np.pi/np.sqrt(2),-np.pi/np.sqrt(2)],[0,0,-np.pi/np.sqrt(2),np.pi/np.sqrt(2)]]
H = [[0,0,0,0],[0,0,0,0],[0,0,1,-1],[0,0,-1,1]]
# init = near_00
# unsafe = out_of_00
def init(x): return [x[0]**2 + x[1]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
init_constraints = [NonlinearConstraint(init, [0.9, 1], [1, 1])]

def unsafe(x): return [x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
unsafe_constraints= [NonlinearConstraint(unsafe, [.11,1], [1,1])]

b_cnot = scipy_find_k_barrier(2, H,
                              init=init_constraints,
                              unsafe=unsafe_constraints,
                              linprog_obj=[40],
                              verbose=1)
b_cnot

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  -1.0*z0*conjugate(z0)
Finding constant...
Checking...
Constant found:  0.9


-1.0*z0*conjugate(z0) + 0.9

$0.9 - 1.0 z_0 \overline{z_0}$

In [25]:
# H = [[0,0,0,0],[0,0,0,0],[0,0,np.pi/np.sqrt(2),-np.pi/np.sqrt(2)],[0,0,-np.pi/np.sqrt(2),np.pi/np.sqrt(2)]]
H = [[0,0,0,0],[0,0,0,0],[0,0,1,-1],[0,0,-1,1]]
# init = near_00_or_01
# unsafe = in_1xregion
def init(x): return [x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
init_constraints = [NonlinearConstraint(init, [0.9, 1], [1, 1])]

def unsafe(x): return [x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
unsafe_constraints= [NonlinearConstraint(unsafe, [.11,1], [1,1])]

b_cnot = scipy_find_k_barrier(2, H,
                              init=init_constraints,
                              unsafe=unsafe_constraints,
                              linprog_obj=[40,31],
                              verbose=1)
b_cnot

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  -1.0*z0*conjugate(z0) - 1.0*z1*conjugate(z1)
Finding constant...
Checking...
Constant found:  0.9


-1.0*z0*conjugate(z0) - 1.0*z1*conjugate(z1) + 0.9

Barrier: $
\displaystyle 0.9 - 1.0 z_0 \overline{z_0} - 1.0 z_1 \overline{z_1}
$

In [24]:
# H = [[0,0,0,0],[0,0,0,0],[0,0,np.pi/np.sqrt(2),-np.pi/np.sqrt(2)],[0,0,-np.pi/np.sqrt(2),np.pi/np.sqrt(2)]]
H = [[0,0,0,0],[0,0,0,0],[0,0,1,-1],[0,0,-1,1]]
# For CNOT
# c[16] = 1
# c[23] = 1
# c[-14] = 1
# c[-5] = 1

# init = near_10
# unsafe = in_0region
def init(x): return [x[0]**2 + x[1]**2,
                     x[2]**2 + x[3]**2,
                     x[4]**2 + x[5]**2,
                     x[6]**2 + x[7]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
init_constraints = [NonlinearConstraint(init, [0, 0, 0.81, 0, 1], [0.09, 0.01, 1, 0.09, 1])]

def unsafe(x): return [x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2,
                     x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
unsafe_constraints= [NonlinearConstraint(unsafe, [.5,0,1], [1,.5,1])]

b_cnot = scipy_find_k_barrier(2, H,
                              init=init_constraints,
                              unsafe=unsafe_constraints,
                              linprog_obj=[16,23],
                              # linprog_obj=list(range(45)),
                              verbose=1)
b_cnot

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  -1.0*z2*conjugate(z2) - 1.0*z3*conjugate(z3)
Finding constant...
Checking...
Constant found:  0.9


-1.0*z2*conjugate(z2) - 1.0*z3*conjugate(z3) + 0.9

Barrier found:
$
\displaystyle 0.9 - 1.0 z_{2} \overline{z_{2}} - 1.0 z_{3} \overline{z_{3}}
$

In [30]:
# H = [[0,0,0,0],[0,0,0,0],[0,0,np.pi/np.sqrt(2),-np.pi/np.sqrt(2)],[0,0,-np.pi/np.sqrt(2),np.pi/np.sqrt(2)]]
H = [[0,0,0,0],[0,0,0,0],[0,0,1,-1],[0,0,-1,1]]

# init = near_10
# unsafe = in_0region
def init(x): return [x[4]**2 + x[5]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
init_constraints = [NonlinearConstraint(init, [0.9, 1], [1, 1])]

def unsafe(x): return [x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
unsafe_constraints= [NonlinearConstraint(unsafe, [.11,1], [1,1])]

b_cnot = scipy_find_k_barrier(2, H,
                              init=init_constraints,
                              unsafe=unsafe_constraints,
                              linprog_obj=[16,23],
                              # linprog_obj=list(range(45)),
                              verbose=1)
b_cnot

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  -1.0*z2*conjugate(z2) - 1.0*z3*conjugate(z3)
Finding constant...
Checking...
Constant found:  0.9


-1.0*z2*conjugate(z2) - 1.0*z3*conjugate(z3) + 0.9

### Never make a barrier

Unsafe region is in path of evolution from initial region

In [17]:
H = [[0,0,0,0],[0,0,0,0],[0,0,np.pi/np.sqrt(2),-np.pi/np.sqrt(2)],[0,0,-np.pi/np.sqrt(2),np.pi/np.sqrt(2)]]

# init = near_10
# unsafe = near_11
def init(x): return [x[0]**2 + x[1]**2,
                     x[2]**2 + x[3]**2,
                     x[4]**2 + x[5]**2,
                     x[6]**2 + x[7]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
init_constraints = [NonlinearConstraint(init, [0, 0, 0.81, 0, 1], [0.09, 0.01, 1, 0.09, 1])]

def unsafe(x): return [x[6]**2 + x[7]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
unsafe_constraints= [NonlinearConstraint(unsafe, [0.5,1], [1,1])]

try:
    b_cnot = scipy_find_k_barrier(2, H,
                                  init=init_constraints,
                                  unsafe=unsafe_constraints,
                                  linprog_obj=[16,23],
                                  verbose=1)
    print(b_cnot)
except:
    print("Cannot find barrier")

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  -1.0*z2*conjugate(z2) - 1.0*z3*conjugate(z3)
Finding constant...
Checking...
Cannot find barrier


### Slightly Disturbed System
See https://journals.aps.org/pra/pdf/10.1103/PhysRevA.69.052325

In [18]:
d = 0.01
H = [[0,0,0,0],
     [0,0,0,0],
     [0,0,1,-1],
     [0,0,-1,1]]
i, j = 1,2
H[i][j] = d
H[j][i] = d


# init = near_10
# unsafe = in_0region
def init(x): return [x[0]**2 + x[1]**2,
                     x[2]**2 + x[3]**2,
                     x[4]**2 + x[5]**2,
                     x[6]**2 + x[7]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
init_constraints = [NonlinearConstraint(init, [0, 0, 0.81, 0, 1], [0.09, 0.01, 1, 0.09, 1])]

def unsafe(x): return [x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2,
                     x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
unsafe_constraints= [NonlinearConstraint(unsafe, [.5,0,1], [1,.5,1])]

b_ctop = scipy_find_k_barrier(2, H,
                              init=init_constraints,
                              unsafe=unsafe_constraints,
                              # linprog_obj=[16,23],
                              # linprog_obj=[71,130],
                              verbose=1)
b_ctop

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  0
Finding constant...
Checking...


Exception: 0
Error: proposed barrier has part of unsafe in same contour as initial region

## $e^{-iHt} = X$

See https://doi.org/10.1142/S0217979297001143

In [ ]:
H = [[.5,-.5],[-.5,.5]]
# Need suitable init and unsafe regions
def init(x): return [x[0]**2,
                     x[1]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2]
init_constraints = [NonlinearConstraint(init, [.9, 0, 1], [1, 0, 1])]

def unsafe(x): return [x[1],
                       x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2]
unsafe_constraints = [NonlinearConstraint(unsafe, [.6, 1], [1, 1])]

try:
    barrier = scipy_find_k_barrier(2, H,
                                   init=init_constraints,
                                   unsafe=unsafe_constraints,
                                   linprog_obj=[0,1],
                                   # linprog_obj=list(range(14)),
                                   verbose=1)
    print(barrier)
except:
    print("Barrier was in unsafe region")

## $e^{-iHt} = Y$

In [ ]:
H = [[.5, .5j],[-.5j,.5]]
init_constraints = close_to_zero
unsafe_constraints = qubit_simple_constraints((_,_),(_,_))

## $e^{-iHt} = Z$

In [ ]:
H = [[2,0],[0,1]]
init_constraints = close_to_zero
unsafe_constraints = qubit_simple_constraints((0,0.89),(0.11,1))

barrier = scipy_find_k_barrier(2, H,
                               init=init_constraints,
                               unsafe=unsafe_constraints,
                               linprog_obj=[0,12],
                               verbose=1)
barrier

In [ ]:
H = [[2,0],[0,1]]
init_constraints = close_to_one
unsafe_constraints = qubit_simple_constraints((0.11,1),(0,0.89))

barrier = scipy_find_k_barrier(2, H,
                               init=init_constraints,
                               unsafe=unsafe_constraints,
                               linprog_obj=[0,7],
                               verbose=1)
barrier